<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
!pip install wandb
!wandb login

     |████████████████████████████████| 2.0MB 7.1MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 163kB 23.9MB/s 
     |████████████████████████████████| 133kB 24.2MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=5ddb45ae5a25ae1a358d8a5647f9d432dfa6871c1f3d46846a53d1eaecda4b93
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=04d8742ec47d5abfab4dfb2c4438b26bb18977a37b43ad828bbbf0c391183397
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: App

In [69]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tensorflow.keras import initializers
import tensorflow as tf

wandb.init(project="CS6910-assg1", entity="narendv")
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

wandb: Currently logged in as: narendv (use `wandb login --relogin` to force relogin)


In [70]:
def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train = np.transpose((np.array(x_train[:54000],dtype=int).reshape([54000,784]))/255)
  Y_train = np.transpose((np.array(y_train[:54000],dtype=int).reshape([54000,1])))
  X_validation = np.transpose((np.array(x_train[54000:],dtype=int).reshape([6000,784]))/255)
  Y_validation = np.transpose(np.array(y_train[54000:],dtype=int).reshape([6000,1]))
  X_test = np.transpose((np.array(x_test,dtype=int).reshape([10000,784]))/255)
  Y_test = np.transpose(np.array(y_test,dtype=int).reshape([10000,1]))
  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [71]:
def log_images():
  images = []
  labels = []
  for i in range(len(X_train)):
    if len(labels)==len(classes):
      break
    j = random.randint(0,53999)
    if classes[Y_train[0,j]] not in labels:
      images.append(X_train[:,j].reshape([28,28]))
      labels.append(classes[Y_train[0,j]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

Loading data

In [72]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = load_data()

In [73]:
log_images()

Defining basic parameters

In [74]:
N= [0,64,32,16,10]  #Insert the number of neurons in each layer
N[0]=X_train.shape[0]
l = len(N) - 1

Weights initialising

In [75]:
def initialize_weights_random(N):
  np.random.seed(1)
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    W.append((np.random.randn(N[i],N[i-1])))
    b.append(np.zeros((N[i],1)))

  return W,b 

In [76]:
def initialize_weights_xavier(N):
  initializer = tf.keras.initializers.GlorotNormal()
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    values = initializer(shape=(N[i],N[i-1]))
    W.append(np.array(values))
    b.append(np.zeros((N[i],1)))
    del values
  return W,b 

In [77]:
def initialize_weights(N,init):
  if init=='random':
    return initialize_weights_random(N)

  if init=='xavier':
    return initialize_weights_xavier(N)

In [78]:
def one_hot(X,y):
  one_hot_y=np.zeros((10,X.shape[1]))
  for i in range(X.shape[1]):
    one_hot_y[y[0,i],i] = 1
  
  return one_hot_y

Softmax

In [79]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [80]:
def sigmoid(a):
  return 1/(1+np.exp(-a))

Sigmoid derivative

In [81]:
def dSigmoid(a):
  return sigmoid(a)*(1-sigmoid(a))

Tanh activation function

In [82]:
def tanh(a):
  return np.tanh(a)

Derivative of the tanh function

In [83]:
def dTanh(a):
  return 1- np.square(tanh(a))

ReLU

In [84]:
def ReLU(a):
  return np.maximum(0,a)

Derivative of ReLU

In [85]:
def dReLU(a):
  return np.greater(a, 0).astype(int)

In [86]:
def activation(a,act):
  if act=='sigmoid':
    return sigmoid(a)
  if act=='tanh':
    return tanh(a)
  if act=='relu':
    return ReLU(a)

def dActivation(a,act):
  if act=='sigmoid':
    return dSigmoid(a)
  if act=='tanh':
    return dTanh(a)
  if act=='relu':
    return dReLU(a)

Cross Entropy

In [115]:
def calculate_loss(y_hat,Y):
  loss = 0
  for i in range(Y.shape[1]):
    loss -= np.log(y_hat[Y[0,i],i])
  return (loss/Y.shape[1])

In [ ]:
def dLoss(y_hat, X, y):
  return (y_hat - one_hot(X,y))

In [ ]:
def l2_loss(y_hat,Y):
  loss=(np.linalg.norm(y_hat-Y))/Y.shape[1]
  return loss

Feed forward network

In [88]:
def feedforward(X_input, W, b, N, l, act):
  a=[]
  a.append(0)
  h=[]
  h.append(X_input)
  for i in range(1,l):
    a_new=b[i]+np.matmul(W[i],h[i-1])
    #print(a_new)
    h_new=activation(a_new,act)
    #print(h_new)
    a.append(a_new)
    h.append(h_new)

  a_new=b[l]+np.matmul(W[l],h[l-1])
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

In [89]:
W_temp,b_temp=initialize_weights(N,'random')
a_temp,h_temp,y_temp=feedforward(X_train, W_temp, b_temp, N, 3, 'sigmoid')

Backpropogation

In [90]:
def backrop(X,y,y_hat,W,a,h,act):
  grad_W=[]
  grad_b=[]
  grad_a=[]
  grad_h=[]
  
  da = y_hat - one_hot(X,y)
  grad_a.append(da)

  for i in reversed(range(1,l+1)):
    dw = np.matmul(da,np.transpose(h[i-1]))/(X.shape[1])
    db = np.sum(da, axis=1, keepdims=True)/(X.shape[1])
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dActivation(a[i-1],act)
    
    grad_W.append(dw)
    grad_b.append(db)
    grad_a.append(da_prev)
    grad_h.append(dh_prev)
    da=da_prev

  grad_W.append(0)
  grad_b.append(0)
  grad_W.reverse()
  grad_b.reverse()
  grad_a.reverse()
  grad_h.reverse()
  

  return grad_W,grad_b,grad_a,grad_h


Gradient Descent

In [118]:
def gradient_descent(epochs, learn_rate, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      a, h, y_hat = feedforward(X, W, b, N, l,act)
      # if j==n-1:
      #   avg_loss=calculate_loss(y_hat, Y)
      #   wandb.log({"loss": avg_loss, "epoch": i})
      grad_W, grad_b = backrop(Y_train, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        W_new.append(W[k] - (learn_rate*grad_W[k])/batch_size)
        b_new.append(b[k] - (learn_rate*grad_b[k])/batch_size)
      W = W_new
      b = b_new
    y_hat = feedforward(X_train, W, b, N, l,act)[2]
    loss=calculate_loss(y_hat, Y_train)
    y_hat_val = feedforward(X_validation, W, b, N, l,act)[2]
    val_loss=calculate_loss(y_hat_val, Y_validation)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})


  return W, b

Momentum based gradient descent

In [119]:
def mom_grad_descent(epochs, learn_rate, gamma, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l,act)
      if j==n-1:
        avg_loss=calculate_loss(y_hat, Y)
        wandb.log({"loss": avg_loss, "epoch": i})
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = gamma*update_prev_w[k] + learn_rate*grad_W[k]
        update_b[k] = gamma*update_prev_b[k] + learn_rate*grad_b[k]
        W_new.append((W[k] - update_w[k]/batch_size))
        b_new.append((b[k] - update_b[k]/batch_size))
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
    
  return W, b

In [93]:
W, b = mom_grad_descent(10,0.001,0.9,1028,'sigmoid','random',N)

SyntaxError: ignored

Nesterov accelerated gradient descent

In [120]:
def nesterov_grad_descent(epochs, learn_rate, gamma, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.floor(X_train.shape[1]/batch_size)+1).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      if j==n-1:
        avg_loss=calculate_loss(y_hat, Y)
        wandb.log({"loss": avg_loss, "epoch": i})
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = gamma*update_prev_w[k]
        update_b[k] = gamma*update_prev_b[k]
        W_new.append((W[k] - update_w[k]))
        b_new.append((b[k] - update_b[k]))
      W = W_new
      b = b_new
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      for k in range(1,l+1):
        update_w[k] = gamma*update_prev_w[k] + learn_rate*grad_W[k]
        update_b[k] = gamma*update_prev_b[k] + learn_rate*grad_b[k]
        W_new.append((W[k] - update_w[k]))
        b_new.append((b[k] - update_b[k]))
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
  return W, b

RMS Prop

In [121]:
def rms_prop(epochs, learn_rate, beta, epsilon, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.floor(X_train.shape[1]/batch_size)+1).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      if j==n-1:
        avg_loss=calculate_loss(y_hat, Y)
        wandb.log({"loss": avg_loss, "epoch": i})
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = beta*update_prev_w[k] + (1-beta)*np.square(grad_W[k])
        update_b[k] = beta*update_prev_b[k] + (1-beta)*np.square(grad_b[k])
        W_new.append((W[k] - (learn_rate*np.divide(grad_W[k],(np.sqrt(update_w[k]+epsilon))))))
        b_new.append((b[k] - (learn_rate*np.divide(grad_b[k],(np.sqrt(update_b[k]+epsilon))))))
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
  return W, b

In [122]:
def adam(epochs, learn_rate, beta1, beta2, epsilon, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  m_prev_w = []
  m_prev_w.append(0)
  m_prev_b = []
  m_prev_b.append(0)
  v_prev_w = []
  v_prev_w.append(0)
  v_prev_b = []
  v_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    m_prev_w.append(np.zeros(W[i].shape))
    m_prev_b.append(np.zeros(b[i].shape))
    v_prev_w.append(np.zeros(W[i].shape))
    v_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      m_w = []
      m_w.append(0)
      v_w = []
      v_w.append(0)
      m_b = []
      m_b.append(0)
      v_b = []
      v_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      if j==n-1:
        avg_loss=calculate_loss(y_hat, Y)
        wandb.log({"loss": avg_loss, "epoch": i})
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      t = i*n + j+1
      for k in range(1,l+1):
        m_w.append(np.zeros(W[k].shape))
        v_w.append(np.zeros(W[k].shape))
        m_b.append(np.zeros(b[k].shape))
        v_b.append(np.zeros(b[k].shape))
        
        m_w[k] = (beta1*m_prev_w[k] + (1-beta1)*grad_W[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k] + (1-beta2)*np.square(grad_W[k]))/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k] + (1-beta1)*grad_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k] + (1-beta2)*np.square(grad_b[k]))/(1-np.power(beta2,t))
        W_new.append((W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon))))))
        b_new.append((b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))))
        m_prev_w[k] = m_w[k]
        v_prev_w[k] = v_w[k]
        m_prev_b[k] = m_b[k]
        v_prev_b[k] = v_b[k]
      del m_w
      del v_w
      del m_b
      del v_b
      W = W_new
      b = b_new

  return W, b

In [ ]:
W, b = adam(10,0.01,0.9,0.99, 10**-8,32,'sigmoid','random',N)

In [123]:
def nadam(epochs, learn_rate, beta1, beta2, epsilon, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  m_prev_w = []
  m_prev_w.append(0)
  m_prev_b = []
  m_prev_b.append(0)
  v_prev_w = []
  v_prev_w.append(0)
  v_prev_b = []
  v_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    m_prev_w.append(np.zeros(W[i].shape))
    m_prev_b.append(np.zeros(b[i].shape))
    v_prev_w.append(np.zeros(W[i].shape))
    v_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      m_w = []
      m_w.append(0)
      v_w = []
      v_w.append(0)
      m_b = []
      m_b.append(0)
      v_b = []
      v_b.append(0)
      print ('yay')
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      if j==n-1:
        avg_loss=calculate_loss(y_hat, Y)
        wandb.log({"loss": avg_loss, "epoch": i})
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      t = i*n + j+1
      for k in range(1,l+1):
        m_w.append(np.zeros(W[k].shape))
        v_w.append(np.zeros(W[k].shape))
        m_b.append(np.zeros(b[k].shape))
        v_b.append(np.zeros(b[k].shape))
        
        m_w[k] = (beta1*m_prev_w[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k])/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k])/(1-np.power(beta2,t))
        W_new.append((W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon))))))
        b_new.append((b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))))
      W = W_new
      b = b_new
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      for k in range(1,l+1):
        m_w[k] = (beta1*m_prev_w[k] + (1-beta1)*grad_W[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k] + (1-beta2)*np.square(grad_W[k]))/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k] + (1-beta1)*grad_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k] + (1-beta2)*np.square(grad_b[k]))/(1-np.power(beta2,t))
        W_new.append((W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon))))))
        b_new.append((b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))))
        m_prev_w[k] = m_w[k]
        v_prev_w[k] = v_w[k]
        m_prev_b[k] = m_b[k]
        v_prev_b[k] = v_b[k]
      del m_w
      del v_w
      del m_b
      del v_b
      W = W_new
      b = b_new
  return W, b

In [ ]:
W, b = nadam(10, 0.01,0.9,0.99,10**-8,128,'sigmoid','random',N)

yay


ValueError: ignored

In [ ]:
W,b = gradient_descent(10,0.001)

237097.25797024355
236504.65984101634
235915.35319173572
235329.34641918368
234746.64739842658
234167.26348595432
233591.20152293865
233018.46783869923
232449.06825427685
231883.00808620665


In [ ]:
W, b = nesterov_grad_descent(10, 0.001,0.9,1024)

2256.143703729159
2047.4780847643306
1911.0641613991295
1814.9768384497925
1743.8156035174752
1690.3848345853462
1650.2513357999287
1620.0728617417656
1597.1403145157337
1579.266565016062


In [ ]:
W, b = rms_prop(3,0.001,0.9,10**-8,1024,'relu','random',N)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
nan


NameError: ignored

In [ ]:
def grad_descent(W,b,grad_W,grad_b,learn_rate=0.001):
  W_new=[]
  W_new.append(0)
  b_new=[]
  b_new.append(0)
  m=X_train.shape[1]
  for i in range(1,l+1):
    #print(W[i].shape,grad_W[i].shape,b[i].shape,grad_b[i].shape)
    W_new.append(W[i] - (learn_rate*grad_W[i])/m)
    b_new.append(b[i] - (learn_rate*grad_b[i])/m)
  return W_new,b_new


Training

In [ ]:
def training(N,l,epochs,batch_size,learn_rate,X_input,y, act):
  W , b = initialize_weights(N)
  n=(np.floor(X_input.shape[1]/batch_size)+1).astype(int)
  for j in range(epochs):
    for j in range(n):
      X=X_input[:,[j*batch_size,(j+1)*batch_size]]
      y=y[:,[j*batch_size,(j+1)*batch_size]]
      a , h , y_hat = feedforward(X, W, b, N, l, act) 
      grad_W , grad_b = backrop(y,y_hat,W,a,h, act)[0:2]
      W , b = grad_descent(W,b,grad_W,grad_b,learn_rate)
    
    calculate_loss(y_hat,y)
  
  return W,b


In [ ]:
def predict(W,b,X_test,N,l, act):
  y_hat = feedforward(X_test, W, b, N, l, act)[2]
  y_class = np.argmax(y_hat,axis=0).reshape([1,X_test.shape[1]])

  return y_class

In [ ]:
Y_predict = predict(W_new,b_new,X_train,N,l)
print (Y_predict[0,0:4])
print (Y_test[0,0:4])
#sklearn.metrics.accuracy_score(Y_test, Y_predict)

[6 6 7 7]
[9 2 1 1]


In [98]:
sweep_config = {
    'method': 'grid'
    }

In [99]:
parameters_dict = {
    # 'l': {
    #     'values': [3, 4, 5]
    #   },
    # 'size of each layer': {
    #     'values': [32, 64, 128]
    #   },
    # 'weight decay': {
    #     'values': [0, 0.0005, 0.5]
    #   },
    'learn_rate': {
        'values': [1e-3]
      },
    'epochs': {
        'values': [10]
      },
    'batch_size': {
        'values': [16,32,64]
      },
    'init': {
        'values': ['random', 'xavier']
      },
    'optimizer': {
        'values': ['mom', 'nest', 'rms']#'sgd', , 'adam', 'nadam'
      },
    'act': {
          'values': ['sigmoid', 'tanh']#, 'ReLU'
      }
    }

sweep_config['parameters'] = parameters_dict

In [124]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'parameters': {'act': {'values': ['sigmoid', 'tanh']},
                'batch_size': {'values': [16, 32, 64]},
                'epochs': {'values': [10]},
                'init': {'values': ['random', 'xavier']},
                'learn_rate': {'values': [0.001]},
                'optimizer': {'values': ['mom', 'nest', 'rms']}}}


In [125]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-assg1")

Create sweep with ID: uzbp043n
Sweep URL: https://wandb.ai/narendv/CS6910-assg1/sweeps/uzbp043n


In [127]:
wandb.agent(sweep_id, training_sweep)

wandb: Agent Starting Run: zx1ohh1a with config:
wandb: 	act: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	init: random
wandb: 	learn_rate: 0.001
wandb: 	optimizer: mom


loss,1.77842
epoch,10
_runtime,121
_timestamp,1615555105
_step,9


loss,█▅▄▄▃▂▂▂▁▁
epoch,▁▁▁▁▁▁▁▁▁▁
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: fsz6kpkv with config:
wandb: 	act: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	init: random
wandb: 	learn_rate: 0.001
wandb: 	optimizer: nest


wandb: ERROR Run fsz6kpkv errored: ZeroDivisionError('division by zero')
wandb: Agent Starting Run: mqgvn5g5 with config:
wandb: 	act: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	init: random
wandb: 	learn_rate: 0.001
wandb: 	optimizer: rms


...

loss,█▆▅▄▃▃▂▂▁▁
epoch,▁▁▁▁▁▁▁▁▁▁
_runtime,▁▂▃▃▄▅▆▇▇█
_timestamp,▁▂▃▃▄▅▆▇▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: johy31c2 with config:
wandb: 	act: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	init: xavier
wandb: 	learn_rate: 0.001
wandb: 	optimizer: rms


loss,0.23351
epoch,10
_runtime,57
_timestamp,1615557376
_step,9


loss,█▇▅▄▄▃▂▁▁▁
epoch,▁▁▁▁▁▁▁▁▁▁
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [126]:
def training_sweep(config=None):
  
  # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        if config.optimizer == 'sgd':
          gradient_descent(config.epochs, config.learn_rate, config.batch_size, config.act, config.init, N)
        if config.optimizer == 'mom':
          mom_grad_descent(config.epochs, config.learn_rate, 0.9, config.batch_size, config.act, config.init, N)
        if config.optimizer == 'nest':
          nesterov_grad_descent(config.epochs, config.learn_rate, 0.9, config.batch_size, config.act, config.init, N)
        if config.optimizer == 'rms':
          rms_prop(config.epochs, config.learn_rate, 0.9, 10**-8, config.batch_size, config.act, config.init, N)
        if config.optimizer == 'adam':
          adam(config.epochs, config.learn_rate, 0.9, 0.99, 10**-8, config.batch_size, config.act, config.init, N)
        if config.optimizer == 'nadam':
          nadam(config.epochs, config.learn_rate, 0.9, 0.99, 10**-8, config.batch_size, config.act, config.init, N)

        #wandb.log({"loss": avg_loss, "epoch": epoch})